## MJ code

In [3]:
# <Save_C3DFeatures_32Segments.m> -> python file


#-*- encoding: utf8 -*-
import os
import numpy as np
import struct

    ## <read_featureFC_file.py>(from 도롱빈)================================
def read_bin(input_file):
# input_file = open(r'.\normal_video_001\000032.fc6-1','rb')
    input_file = open(input_file,'rb')
    try:
        sizes = [struct.unpack('i',input_file.read(4))[0] for i in range(5)]
        m = np.prod(sizes)
        data = [struct.unpack('f',input_file.read(4))[0] for i in range(m)]
    finally:
        input_file
    feature_vector = np.array(data)

    return feature_vector, feature_vector.shape

    ## ======================================================================

C3D_Path = r'/content/drive/My Drive/warming-up project/c3d_ex/SampleOutputs'
C3D_Path_Seg = r'/content/drive/My Drive/warming-up project/c3d_ex/SampleOutputs_final_c3d'


All_Folder = os.listdir(C3D_Path)
subcript = '_C_mj.txt'

for ifolder in All_Folder:
    print(ifolder)
    # START 1 LOOP WITH 1 FC FOLDER(영상 1개)

    Folder_Path = str(C3D_Path) + "/" + str(ifolder)
    AllFiles = os.listdir(Folder_Path) # /.fc6-1 확장자 파일 싹다 리스트로 반환
                                       # fc6-1 files in feature directory, each file = a clip in video
                                       # one clip = 16 frames

    if len(AllFiles) == 0:
        print("no fc6-1 file in path")
        continue
    
    # .fc6-1 읽고 feature_vector 생성
    feature_vector = np.zeros((len(AllFiles), 4096))
    # each fc6-1 = 1 clips 16 frames = 4096-d ==> Total is [N/16]=88 clips like that
    # Iterate each fc6-1 file (16 frames each)
    for ifile in range(0,len(AllFiles)):
        FilePath =Folder_Path + '/' + AllFiles[ifile]

        data, s = read_bin(FilePath)
        feature_vector[ifile]=data # 1 row 4096-d in 88x4096 is assign by 1 clip feature (4096) ???????????? column을 row로 바꿈

    # 이제 fc6-1을 모두 읽어서 feature_vector(shape = (fc6-1 files 개수) x 4096)에 넣음


#----------------------------------------------------------------------------------
# 여기서부터 feature vector를 32seg로 나눔
# ((fc6-1 files 개수), 4096) -> (32, 4096)으로 만들어서 새로운 파일로 저장
# 여기서 만든 파일을 Training_AnomalyDetector_public.py에 넣어서 training 시킬 수 있음

    txt_file_name = C3D_Path_Seg + '/' + ifolder +subcript
    # feature txt name i.e abnormal_video_001_C.txt

    # fid1 = open(txt_file_name, 'w')

    sum_row = np.sum(feature_vector, axis=1) # 각 16 frame clip에 해당하는 row를 sum
    if np.where(sum_row==0)[0].size != 0:
        print('Error2')
    if np.isnan(feature_vector).sum() != 0:
        print('Error3')
    if np.inf in feature_vector:
        print('Error4')
 

    # 32 Segments
    Segments_Features = np.zeros((32, 4096))    # (32, 4096)
    thirty2_shots = np.linspace(0, len(AllFiles)-1, 33).astype(int) # AllFiles = .fc-1 확장자 파일들 ?????함수 안의 값 변경
    # SO: thirty2shots = [1 , 4, 6, 10, ..... 83, 85, 88], total elements is 33, vector 1x33
    
    count = -1
    # WRITE (88, 4096) TO (32, 4096)
    for ishots in range(0,len(thirty2_shots) - 1):  # ishorts starts from 0 to 31 (총 32개)
        ss = int(thirty2_shots[ishots])            # start clip index 
        ee = int(thirty2_shots[ishots + 1] - 1)     # end clip index 
        

        if ss == ee: # thirty2_shots에서 값의 차이가 1일 때
            temp_vect = feature_vector[ss] # ss==ee --> get 1 vector 4096-d from 88x4096

        elif ee < ss: # 영상 frame이 적어서 thirty2_shots에 같은 숫자가 나오는 경우
            temp_vect = feature_vector[ss]  # get 1 vector 4096-d from 88x4096

        else:
            temp_vect = feature_vector[ss:ee].mean(axis=0) # 각 열의 평균값을 가진 (1, 4096) 추출
                                                           # multiple rows vector -> one row vector

        # NORM-2 (L2)
        temp_vect = np.round(temp_vect / np.linalg.norm(temp_vect), 6)
        # divide by norm-2 (L2) of vector (Euclidean norm)=cumsum(sqrt(x[i]^2))

        if np.linalg.norm(temp_vect) == 0:
            print('Error5')

        count = count + 1 # next segment (max=32)
        Segments_Features[count]= temp_vect # push each segment feature to final 32 video segments feature



    # save 32 segment features in text file ( You can directly save and load .mat file in python as well).
    print(Segments_Features)
    print(Segments_Features.shape)

    # for i in range(0,Segments_Features.shape[0]):
    #     feat_text = str(Segments_Features[i].tolist())
    #     fid1.write(feat_text)
    #     fid1.write('\n')
    np.savetxt(txt_file_name, Segments_Features, fmt="%.6f")

    # fid1.close()



Explosion008_x264
[[0.004291 0.       0.011497 ... 0.       0.       0.020077]
 [0.004544 0.       0.008558 ... 0.       0.       0.023938]
 [0.00186  0.       0.001331 ... 0.       0.       0.021174]
 ...
 [0.004323 0.002312 0.006303 ... 0.       0.036721 0.000178]
 [0.       0.000649 0.       ... 0.       0.001883 0.      ]
 [0.       0.003771 0.       ... 0.       0.       0.      ]]
(32, 4096)
Explosion025_x264
[[0.030636 0.       0.       ... 0.       0.       0.      ]
 [0.030636 0.       0.       ... 0.       0.       0.      ]
 [0.029071 0.       0.       ... 0.       0.       0.      ]
 ...
 [0.       0.       0.       ... 0.013327 0.       0.033512]
 [0.       0.002152 0.007484 ... 0.       0.010756 0.013517]
 [0.       0.002152 0.007484 ... 0.       0.010756 0.013517]]
(32, 4096)
Normal_Videos_006_x264
[[0.       0.       0.       ... 0.       0.       0.007479]
 [0.       0.       0.       ... 0.       0.       0.007479]
 [0.       0.       0.       ... 0.       0.       0.

## SB code

In [2]:
# clc() ##command창에 표시된 코드들 사라지게해
#-*- encoding: utf8 -*-
import os
import numpy as np
import struct
# clear(mstring('all'))
# close(mstring('all'))

# This code save already computed C3D features into 32 (video features) segments.
# We assume that C3D features for a video are already computed. We use
# default settings for computing C3D features, i.e., we compute C3D features for
# every 16 frames and obtain the features from fc6.

import struct

def read_bin(input_file):
# input_file = open(r'.\normal_video_001\000032.fc6-1','rb')
    input_file = open(input_file,'rb')
    try:
        sizes = [struct.unpack('i',input_file.read(4))[0] for i in range(5)]
        m = np.prod(sizes)
        data = [struct.unpack('f',input_file.read(4))[0] for i in range(m)]
    finally:
        input_file
    feature_vector = np.array(data)

    return feature_vector, feature_vector.shape

C3D_Path = r'/content/drive/My Drive/warming-up project/c3d_ex/SampleOutputs'
C3D_Path_Seg = r'/content/drive/My Drive/warming-up project/c3d_ex/SampleOutputs_final_c3d'

if not os.path.isdir(C3D_Path_Seg):
    os.mkdir(C3D_Path_Seg)

print('DONE')

All_Folder = os.listdir(C3D_Path)
# All_Folder = All_Folder[3:end]
subcript = '_C_sb.txt'

for ifolder in All_Folder:
    print(ifolder)
    #% START 1 LOOP WITH 1 FC FOLDER, ex: Abuse028 has N=1392 frames

    Folder_Path = str(C3D_Path) + "/" + str(ifolder)
    #Folder_Path is path of a folder which contains C3D features (for every 16 frames) for a paricular video.
    # N=1392 frames --> it has [1392/16] = 88 fc6-1 files

    AllFiles = os.listdir(Folder_Path) ##"/.fc6-1 확장자 파일 싹다 리스트로 반환"
    # fc6-1 files in feature directory, each file = a clip in video
    # one clip = 16 frames

    if len(AllFiles) == 0:
        print("no fc6-1 file in path")
        continue
        
    feature_vector = np.zeros((len(AllFiles), 4096))
    # each fc6-1 = 1 clips 16 frames = 4096-d ==> Total is [N/16]=88 clips like that
    #% Iterate each fc6-1 file (16 frames each)
    for ifile in range(0,len(AllFiles)):
        FilePath =Folder_Path + '/' + AllFiles[ifile]

        data,_ = read_bin(FilePath)
        _,s = read_bin(FilePath)
        feature_vector[ifile]=data #% 1 column 4096-d in 88x4096 is assign by 1 clip feature (4096)

        # clear(mstring('data')) # clear라는 변수를 매트랩 내에서 삭제

    #% At this point, Feature vector is filled with all actual data from
    # all 16-frame clips in video, each clip is 4096-d, therefore 88x4096
    # is now filled with actual data
    # if sum(sum(feature_vector, [])) == 0: ## 고쳐야됨 : 각 열의 합인 한 행짜리 행렬로
    #     print('error1')

        # Write C3D features in text file to load in
        # Training_AnomalyDetector_public ( You can directly use .mat format if you want).
        txt_file_name = C3D_Path_Seg + '/' + ifolder +subcript
        # feature txt name i.e Abuse028_x264_C.txt

    # if exist(txt_file_name, 'file'):
    #     continue

    fid1 = open(txt_file_name, 'w')
    ## sum(x,1) = sum vertically (column)
    ## sum(x,2) = sum horizontally (row)
    # if not isempty(find(sum(Feature_vect, 2) == 0)): # sum row --> 88x4096 results in 88 rows
    #     # k = find(X,n)은 X의 0이 아닌 요소에 대응하는 처음 n개의 인덱스를 반환합니다
    #     print('error2')


    # if not isempty(find(isnan(Feature_vect(mslice[:])))):
    #     print('error3')

    # if not isempty(find(Feature_vect(mslice[:]) == Inf)):
    #     print('error4')

        #% 32 Segments

    Segments_Features = np.zeros((32, 4096))    #32 row, 4096 column
    thirty2_shots = np.linspace(1, len(AllFiles), 33).round(0)
    # thirty2shots = divide 88 frames to 33 segment, start from 1 to 88
    # SO: thirty2shots = [1 , 4, 6, 10, ..... 83, 85, 88], total elements
    # is 33, vector 1x33
    count = -1
    #% WRITE 88x4096 TO 32x4096
    for ishots in range(0,len(thirty2_shots) - 1):                                   # ishorts starts from 1 to 32
        ss = int(thirty2_shots[ishots] )                                   # start clip index in 88x4096
        ee = int(thirty2_shots[ishots + 1] - 1)                             # end clip index in 88x4096
        
        # print(ss,ee,'llllll')
        if ishots == len(thirty2_shots):
            ee = int(thirty2_shots[ishots + 1])
            #% THIS IS A FEATURE FOR 1 SEGMENT
            #ALL BELOW CASE, temp_vect is always 4096-d based on value of start ss and end ee index
        if ss == ee:
            temp_vect = feature_vector[ss] ##ss번째 행 벡터 추출     # ss==ee --> get 1 vector 4096-d from 88x4096

        elif ee < ss:
            temp_vect = feature_vector[ss]  # ee < ss --> get 1 vector 4096-d from 88x4096

        else:
            temp_vect = feature_vector[ss:ee].mean(axis=0) ##각 열의 평균값을 가진 1*4096행 추출
            # for i in range(ss,ee):
                # feature_vector
            # ss < ee --> get all clip vectors from ss to ee (ex: 3 vectors) from 88x4096
            # origin feature, than take mean value of all (i.e 3 vectors) that vectors to
            # get a new one has 4096-d (shape of result is shape of row when get mean a
            # matrix)
            #mean a vector = mean of each column = sum column/total row -->
            #shape = number of row (=4096) after this mean operation
        # print(temp_vect.shape)
        #% AFTER HAS 1 SEGMENT FEATURE,  CALCULATE NORM-2 (L2)
        temp_vect = temp_vect / np.linalg.norm(temp_vect)
        # temp_vect = temp_vect / np.norm(temp_vect)   #% divide by norm-2 (L2) of vector (Euclidean norm)=cumsum(sqrt(x[i]^2))                                     # divide by norm-2 (L2) of vector (Euclidean norm)=cumsum(sqrt(x[i]^2))

        # if np.linalg.norm(temp_vect) == 0:
        #     print('error5')

        count = count + 1 # next segment (max=32)
        Segments_Features[count]= temp_vect # push each segment feature to final 32 video segments feature

#verify

    # if not isempty(find(sum(Segments_Features, 2) == 0)):
    #     print('error6')

    # if not isempty(find(isnan(Segments_Features(mslice[:])))):
    #     print('error7')


    # if not isempty(find(Segments_Features(mslice[:]) == Inf)):
    #     print('error8')


        # save 32 segment features in text file ( You can directly save and load .mat file in python as well).
    print(Segments_Features)
    print(Segments_Features.shape)

    for i in range(0,Segments_Features.shape[0]):
        feat_text = str(Segments_Features[i].tolist())
        fid1.write(feat_text)
        fid1.write('\n')

    fid1.close()



DONE
[[0.00325776 0.         0.01111239 ... 0.         0.         0.0209416 ]
 [0.00399411 0.         0.00750267 ... 0.         0.         0.01971691]
 [0.         0.         0.00453475 ... 0.         0.         0.02417561]
 ...
 [0.00230024 0.         0.         ... 0.         0.0378602  0.        ]
 [0.         0.00570194 0.         ... 0.         0.00154962 0.        ]
 [0.         0.         0.         ... 0.         0.00159626 0.        ]]
(32, 4096)
[[0.02907103 0.         0.         ... 0.         0.         0.        ]
 [0.03017978 0.         0.         ... 0.         0.         0.        ]
 [0.03007346 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.00215197 0.00748449 ... 0.         0.0107563  0.01351749]
 [0.         0.00215197 0.00748449 ... 0.         0.0107563  0.01351749]
 [0.         0.00215197 0.00748449 ... 0.         0.0107563  0.01351749]]
(32, 4096)
[[0.         0.         0.         ... 0.         0.         0.00720029]
 [0.        

CH code

In [33]:
# clc() ##command창에 표시된 코드들 사라지게해
#-*- encoding: utf8 -*-
import os
import numpy as np
import struct
# clear(mstring('all'))
# close(mstring('all'))

# This code save already computed C3D features into 32 (video features) segments.
# We assume that C3D features for a video are already computed. We use
# default settings for computing C3D features, i.e., we compute C3D features for
# every 16 frames and obtain the features from fc6.

import struct

def read_bin(input_file):
# input_file = open(r'.\normal_video_001\000032.fc6-1','rb')
    input_file = open(input_file,'rb')
    try:
        sizes = [struct.unpack('i',input_file.read(4))[0] for i in range(5)]
        m = np.prod(sizes)
        data = [struct.unpack('f',input_file.read(4))[0] for i in range(m)]
    finally:
        input_file
    feature_vector = np.array(data)

    return feature_vector, feature_vector.shape

C3D_Path = r'/content/drive/My Drive/warming-up project/c3d_ex/SampleOutputs'
C3D_Path_Seg = r'/content/drive/My Drive/warming-up project/c3d_ex/SampleOutputs_final_c3d'

if not os.path.isdir(C3D_Path_Seg):
    os.mkdir(C3D_Path_Seg)


All_Folder = os.listdir(C3D_Path)
# All_Folder = All_Folder[3:end]
subcript = '_C_ch.txt'

for ifolder in All_Folder:
    print(ifolder)
    #% START 1 LOOP WITH 1 FC FOLDER, ex: Abuse028 has N=1392 frames

    Folder_Path = str(C3D_Path) + "/" + str(ifolder)
    #Folder_Path is path of a folder which contains C3D features (for every 16 frames) for a paricular video.
    # N=1392 frames --> it has [1392/16] = 88 fc6-1 files

    AllFiles = os.listdir(Folder_Path) ##"/.fc6-1 확장자 파일 싹다 리스트로 반환"
    # fc6-1 files in feature directory, each file = a clip in video
    # one clip = 16 frames

    if len(AllFiles) == 0:
        print("no fc6-1 file in path")
        continue
        
    feature_vector = np.zeros((len(AllFiles), 4096))
    # each fc6-1 = 1 clips 16 frames = 4096-d ==> Total is [N/16]=88 clips like that
    #% Iterate each fc6-1 file (16 frames each)
    for ifile in range(0,len(AllFiles)):
        FilePath =Folder_Path + '/' + AllFiles[ifile]

        data,_ = read_bin(FilePath)
        _,s = read_bin(FilePath)
        feature_vector[ifile]=data; #% 1 column 4096-d in 88x4096 is assign by 1 clip feature (4096)

        # clear(mstring('data')) # clear라는 변수를 매트랩 내에서 삭제

    #% At this point, Feature vector is filled with all actual data from
    # all 16-frame clips in video, each clip is 4096-d, therefore 88x4096
    # is now filled with actual data
    # if sum(sum(feature_vector, [])) == 0: ## 고쳐야됨 : 각 열의 합인 한 행짜리 행렬로
    #     print('error1')

        # Write C3D features in text file to load in
        # Training_AnomalyDetector_public ( You can directly use .mat format if you want).
        txt_file_name = C3D_Path_Seg + '/' + ifolder +subcript
        # feature txt name i.e Abuse028_x264_C.txt

    # if exist(txt_file_name, 'file'):
    #     continue

    fid1 = open(txt_file_name, 'w')
    ## sum(x,1) = sum vertically (column)
    ## sum(x,2) = sum horizontally (row)
    # if not isempty(find(sum(Feature_vect, 2) == 0)): # sum row --> 88x4096 results in 88 rows
    #     # k = find(X,n)은 X의 0이 아닌 요소에 대응하는 처음 n개의 인덱스를 반환합니다
    #     print('error2')


    # if not isempty(find(isnan(Feature_vect(mslice[:])))):
    #     print('error3')

    # if not isempty(find(Feature_vect(mslice[:]) == Inf)):
    #     print('error4')

        #% 32 Segments

    Segments_Features = np.zeros((32, 4096))    #32 row, 4096 column
    thirty2_shots = np.linspace(1, len(AllFiles), 33).round(0)
    # thirty2shots = divide 88 frames to 33 segment, start from 1 to 88
    # SO: thirty2shots = [1 , 4, 6, 10, ..... 83, 85, 88], total elements
    # is 33, vector 1x33
    count = -1
    #% WRITE 88x4096 TO 32x4096
    for ishots in range(0,len(thirty2_shots) - 1):                                   # ishorts starts from 1 to 32
        ss = int(thirty2_shots[ishots] )                                   # start clip index in 88x4096
        ee = int(thirty2_shots[ishots + 1] - 1)                             # end clip index in 88x4096
        
        # print(ss,ee,'llllll')
        if ishots == len(thirty2_shots):
            ee = int(thirty2_shots[ishots + 1])
            #% THIS IS A FEATURE FOR 1 SEGMENT
            #ALL BELOW CASE, temp_vect is always 4096-d based on value of start ss and end ee index
        if ss == ee:
            temp_vect = feature_vector[ss] ##ss번째 행 벡터 추출     # ss==ee --> get 1 vector 4096-d from 88x4096

        elif ee < ss:
            temp_vect = feature_vector[ss]  # ee < ss --> get 1 vector 4096-d from 88x4096

        else:
            temp_vect = feature_vector[ss:ee].mean(axis=0) ##각 열의 평균값을 가진 1*4096행 추출
            # for i in range(ss,ee):
                # feature_vector
            # ss < ee --> get all clip vectors from ss to ee (ex: 3 vectors) from 88x4096
            # origin feature, than take mean value of all (i.e 3 vectors) that vectors to
            # get a new one has 4096-d (shape of result is shape of row when get mean a
            # matrix)
            #mean a vector = mean of each column = sum column/total row -->
            #shape = number of row (=4096) after this mean operation
        # print(temp_vect.shape)
        #% AFTER HAS 1 SEGMENT FEATURE,  CALCULATE NORM-2 (L2)
        temp_vect = temp_vect / np.linalg.norm(temp_vect)
        # temp_vect = temp_vect / np.norm(temp_vect)   #% divide by norm-2 (L2) of vector (Euclidean norm)=cumsum(sqrt(x[i]^2))                                     # divide by norm-2 (L2) of vector (Euclidean norm)=cumsum(sqrt(x[i]^2))

        # if np.linalg.norm(temp_vect) == 0:
        #     print('error5')

        count = count + 1 # next segment (max=32)
        Segments_Features[count]= temp_vect # push each segment feature to final 32 video segments feature

#verify

    # if not isempty(find(sum(Segments_Features, 2) == 0)):
    #     print('error6')

    # if not isempty(find(isnan(Segments_Features(mslice[:])))):
    #     print('error7')


    # if not isempty(find(Segments_Features(mslice[:]) == Inf)):
    #     print('error8')


        # save 32 segment features in text file ( You can directly save and load .mat file in python as well).
    print(Segments_Features)
    print(Segments_Features.shape)

    for i in range(0,Segments_Features.shape[0]):
        feat_text = str(Segments_Features[i].tolist())
        fid1.write(feat_text)
        fid1.write('\n')

    fid1.close()

Explosion008_x264
[[0.00325776 0.         0.01111239 ... 0.         0.         0.0209416 ]
 [0.00399411 0.         0.00750267 ... 0.         0.         0.01971691]
 [0.         0.         0.00453475 ... 0.         0.         0.02417561]
 ...
 [0.00230024 0.         0.         ... 0.         0.0378602  0.        ]
 [0.         0.00570194 0.         ... 0.         0.00154962 0.        ]
 [0.         0.         0.         ... 0.         0.00159626 0.        ]]
(32, 4096)
Explosion025_x264
[[0.02907103 0.         0.         ... 0.         0.         0.        ]
 [0.03017978 0.         0.         ... 0.         0.         0.        ]
 [0.03007346 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.00215197 0.00748449 ... 0.         0.0107563  0.01351749]
 [0.         0.00215197 0.00748449 ... 0.         0.0107563  0.01351749]
 [0.         0.00215197 0.00748449 ... 0.         0.0107563  0.01351749]]
(32, 4096)
Normal_Videos_006_x264
[[0.         0.         0.     

# 코드 비교하기

In [34]:
import pandas as pd
explosion008_sb = pd.read_csv(r'/content/drive/My Drive/warming-up project/c3d_ex/SampleOutputs_final_c3d/Explosion008_x264_C_sb.txt', delimiter=' ', header=None)
explosion008_mj = pd.read_csv(r'/content/drive/My Drive/warming-up project/c3d_ex/SampleOutputs_final_c3d/Explosion008_x264_C_mj.txt', delimiter=' ', header=None)
explosion008_ch = pd.read_csv(r'/content/drive/My Drive/warming-up project/c3d_ex/SampleOutputs_final_c3d/Explosion008_x264_C_ch.txt', delimiter=' ', header=None)
explosion008 = pd.read_csv(r'/content/drive/My Drive/warming-up project/c3d_ex/SampleOutputs_final_c3d/Explosion008_x264.txt', delimiter=' ', header=None)

In [36]:
sb_exp008 = [i.replace('[', '')[:8] for i in explosion008_sb.iloc[:, 0].values]
ch_exp008 = [i.replace('[', '')[:8] for i in explosion008_ch.iloc[:, 0].values]
mj_exp008 = [format(i, '.6f') for i in explosion008_mj.iloc[:, 0].values]
exp008 = [format(i, '.6f') for i in explosion008.iloc[:, 0].values]
pd.DataFrame({'sb_exp008': sb_exp008, 'ch_exp008':ch_exp008, 'mj_exp008': mj_exp008, 'exp008': exp008})

,sb_exp008,ch_exp008,mj_exp008,exp008
0,0.003257,0.003257,0.004291,0.004290
1,0.003994,0.003994,0.004544,0.003822
2,"0.0,","0.0,",0.001860,0.000910
3,0.005045,0.005045,0.003110,0.002630
4,0.001080,0.001080,0.000128,0.001081
5,"0.0,","0.0,",0.002789,0.002823
6,"0.0,","0.0,",0.000000,0.000000
7,"0.0,","0.0,",0.000000,0.000000
8,0.001033,0.001033,0.000000,0.000692
9,"0.0,","0.0,",0.001733,0.000861


In [42]:
# MJ code 방식
np.linspace(0, 87, 33).astype(int) + 1 # Sultani : [1 , 4, 6, 10, ..... 83, 85, 88]

array([ 1,  3,  6,  9, 11, 14, 17, 20, 22, 25, 28, 30, 33, 36, 39, 41, 44,
       47, 49, 52, 55, 58, 60, 63, 66, 68, 71, 74, 77, 79, 82, 85, 88])

In [46]:
# SB, CH code 방식
np.linspace(1, 88, 33).round(0)

array([ 1.,  4.,  6.,  9., 12., 15., 17., 20., 23., 25., 28., 31., 34.,
       36., 39., 42., 44., 47., 50., 53., 55., 58., 61., 64., 66., 69.,
       72., 74., 77., 80., 83., 85., 88.])

In [44]:
# MJ + SB, CH 방식
np.linspace(0, 87, 33).round(0).astype(int) + 1 

array([ 1,  4,  6,  9, 12, 15, 17, 20, 23, 25, 28, 31, 34, 36, 39, 42, 45,
       47, 50, 53, 55, 58, 61, 64, 66, 69, 72, 74, 77, 80, 83, 85, 88])

### 32-seg로 나누는 방식이 matlab과 달라서 술타니가 준 값과 다를 수 밖에 없음


# 합친 방식
- thiry3_shots 부분을 합친 방식으로 사용

In [48]:
# <Save_C3DFeatures_32Segments.m> -> python file


#-*- encoding: utf8 -*-
import os
import numpy as np
import struct

    ## <read_featureFC_file.py>(from 도롱빈)================================
def read_bin(input_file):
# input_file = open(r'.\normal_video_001\000032.fc6-1','rb')
    input_file = open(input_file,'rb')
    try:
        sizes = [struct.unpack('i',input_file.read(4))[0] for i in range(5)]
        m = np.prod(sizes)
        data = [struct.unpack('f',input_file.read(4))[0] for i in range(m)]
    finally:
        input_file
    feature_vector = np.array(data)

    return feature_vector, feature_vector.shape

    ## ======================================================================

C3D_Path = r'/content/drive/My Drive/warming-up project/c3d_ex/SampleOutputs'
C3D_Path_Seg = r'/content/drive/My Drive/warming-up project/c3d_ex/SampleOutputs_final_c3d'


All_Folder = os.listdir(C3D_Path)
subcript = '_C.txt'

for ifolder in All_Folder:
    print(ifolder)
    # START 1 LOOP WITH 1 FC FOLDER(영상 1개)

    Folder_Path = str(C3D_Path) + "/" + str(ifolder)
    AllFiles = os.listdir(Folder_Path) # /.fc6-1 확장자 파일 싹다 리스트로 반환
                                       # fc6-1 files in feature directory, each file = a clip in video
                                       # one clip = 16 frames

    if len(AllFiles) == 0:
        print("no fc6-1 file in path")
        continue
    
    # .fc6-1 읽고 feature_vector 생성
    feature_vector = np.zeros((len(AllFiles), 4096))
    # each fc6-1 = 1 clips 16 frames = 4096-d ==> Total is [N/16]=88 clips like that
    # Iterate each fc6-1 file (16 frames each)
    for ifile in range(0,len(AllFiles)):
        FilePath =Folder_Path + '/' + AllFiles[ifile]

        data, s = read_bin(FilePath)
        feature_vector[ifile]=data # 1 row 4096-d in 88x4096 is assign by 1 clip feature (4096) 

    # 이제 fc6-1을 모두 읽어서 feature_vector(shape = (fc6-1 files 개수) x 4096)에 넣음


#----------------------------------------------------------------------------------
# 여기서부터 feature vector를 32seg로 나눔
# ((fc6-1 files 개수), 4096) -> (32, 4096)으로 만들어서 새로운 파일로 저장
# 여기서 만든 파일을 Training_AnomalyDetector_public.py에 넣어서 training 시킬 수 있음

    txt_file_name = C3D_Path_Seg + '/' + ifolder +subcript
    # feature txt name i.e abnormal_video_001_C.txt

    # fid1 = open(txt_file_name, 'w')

    sum_row = np.sum(feature_vector, axis=1) # 각 16 frame clip에 해당하는 row를 sum
    if np.where(sum_row==0)[0].size != 0:
        print('Error2')
    if np.isnan(feature_vector).sum() != 0:
        print('Error3')
    if np.inf in feature_vector:
        print('Error4')
 

    # 32 Segments
    Segments_Features = np.zeros((32, 4096))    # (32, 4096)
    thirty2_shots = np.linspace(0, len(AllFiles)-1, 33).round(0).astype(int) # AllFiles = .fc-1 확장자 파일들
    # SO: thirty2shots = [1 , 4, 6, 10, ..... 83, 85, 88], total elements is 33, vector 1x33
    
    count = -1
    # WRITE (88, 4096) TO (32, 4096)
    for ishots in range(0,len(thirty2_shots) - 1):  # ishorts starts from 0 to 31 (총 32개)
        ss = int(thirty2_shots[ishots])            # start clip index 
        ee = int(thirty2_shots[ishots + 1] - 1)     # end clip index 
        

        if ss == ee: # thirty2_shots에서 값의 차이가 1일 때
            temp_vect = feature_vector[ss] # ss==ee --> get 1 vector 4096-d from 88x4096

        elif ee < ss: # 영상 frame이 적어서 thirty2_shots에 같은 숫자가 나오는 경우
            temp_vect = feature_vector[ss]  # get 1 vector 4096-d from 88x4096

        else:
            temp_vect = feature_vector[ss:ee].mean(axis=0) # 각 열의 평균값을 가진 (1, 4096) 추출
                                                           # multiple rows vector -> one row vector

        # NORM-2 (L2)
        temp_vect = np.round(temp_vect / np.linalg.norm(temp_vect), 6)
        # divide by norm-2 (L2) of vector (Euclidean norm)=cumsum(sqrt(x[i]^2))

        if np.linalg.norm(temp_vect) == 0:
            print('Error5')

        count = count + 1 # next segment (max=32)
        Segments_Features[count]= temp_vect # push each segment feature to final 32 video segments feature



    # save 32 segment features in text file ( You can directly save and load .mat file in python as well).
    print(Segments_Features)
    print(Segments_Features.shape)

    # for i in range(0,Segments_Features.shape[0]):
    #     feat_text = str(Segments_Features[i].tolist())
    #     fid1.write(feat_text)
    #     fid1.write('\n')
    np.savetxt(txt_file_name, Segments_Features, fmt="%.6f")

    # fid1.close()



Explosion008_x264
[[0.004291 0.       0.011497 ... 0.       0.       0.020077]
 [0.004143 0.       0.008687 ... 0.       0.       0.021771]
 [0.001365 0.       0.       ... 0.       0.       0.023503]
 ...
 [0.       0.       0.       ... 0.       0.047104 0.000232]
 [0.       0.001721 0.       ... 0.       0.00151  0.      ]
 [0.       0.       0.       ... 0.       0.       0.      ]]
(32, 4096)
Explosion025_x264
[[0.030636 0.       0.       ... 0.       0.       0.      ]
 [0.029071 0.       0.       ... 0.       0.       0.      ]
 [0.03018  0.       0.       ... 0.       0.       0.      ]
 ...
 [0.       0.       0.       ... 0.013327 0.       0.033512]
 [0.       0.002152 0.007484 ... 0.       0.010756 0.013517]
 [0.       0.002152 0.007484 ... 0.       0.010756 0.013517]]
(32, 4096)
Normal_Videos_006_x264
[[0.       0.       0.       ... 0.       0.       0.007479]
 [0.       0.       0.       ... 0.       0.       0.0072  ]
 [0.       0.       0.       ... 0.       0.       0.

In [49]:
import pandas as pd
explosion008_sb = pd.read_csv(r'/content/drive/My Drive/warming-up project/c3d_ex/SampleOutputs_final_c3d/Explosion008_x264_C_sb.txt', delimiter=' ', header=None)
explosion008_ch = pd.read_csv(r'/content/drive/My Drive/warming-up project/c3d_ex/SampleOutputs_final_c3d/Explosion008_x264_C_ch.txt', delimiter=' ', header=None)
explosion008_mj = pd.read_csv(r'/content/drive/My Drive/warming-up project/c3d_ex/SampleOutputs_final_c3d/Explosion008_x264_C_mj.txt', delimiter=' ', header=None)
explosion008_final = pd.read_csv(r'/content/drive/My Drive/warming-up project/c3d_ex/SampleOutputs_final_c3d/Explosion008_x264_C.txt', delimiter=' ', header=None)
explosion008 = pd.read_csv(r'/content/drive/My Drive/warming-up project/c3d_ex/SampleOutputs_final_c3d/Explosion008_x264.txt', delimiter=' ', header=None)

# 1st column
sb_exp008 = [i.replace('[', '')[:8] for i in explosion008_sb.iloc[:, 0].values]
ch_exp008 = [i.replace('[', '')[:8] for i in explosion008_ch.iloc[:, 0].values]
mj_exp008 = [format(i, '.6f') for i in explosion008_mj.iloc[:, 0].values]
final_exp008 = [format(i, '.6f') for i in explosion008_final.iloc[:, 0].values]
exp008 = [format(i, '.6f') for i in explosion008.iloc[:, 0].values]
pd.DataFrame({'sb_exp008': sb_exp008, 'ch_exp008':ch_exp008, 'mj_exp008': mj_exp008, 'final_exp008': final_exp008,'exp008': exp008})

,sb_exp008,ch_exp008,mj_exp008,final_exp008,exp008
0,0.003257,0.003257,0.004291,0.004291,0.004290
1,0.003994,0.003994,0.004544,0.004143,0.003822
2,"0.0,","0.0,",0.001860,0.001365,0.000910
3,0.005045,0.005045,0.003110,0.003548,0.002630
4,0.001080,0.001080,0.000128,0.001612,0.001081
5,"0.0,","0.0,",0.002789,0.004192,0.002823
6,"0.0,","0.0,",0.000000,0.000000,0.000000
7,"0.0,","0.0,",0.000000,0.000000,0.000000
8,0.001033,0.001033,0.000000,0.000000,0.000692
9,"0.0,","0.0,",0.001733,0.001154,0.000861


In [71]:
# 1st-row, last-row
pd.set_option('display.max_rows', None)
sb_exp008 = [i.replace('[', '')[:8] for i in explosion008_sb.iloc[[0, -1], :].values.flatten().tolist()]
ch_exp008 = [i.replace('[', '')[:8] for i in explosion008_ch.iloc[[0, -1], :].values.flatten().tolist()]
mj_exp008 = [format(i, '.6f') for i in explosion008_mj.iloc[[0, -1], :].values.flatten()]
final_exp008 = [format(i, '.6f') for i in explosion008_final.iloc[[0, -1], :].values.flatten()]
exp008 = [format(i, '.6f') for i in explosion008.iloc[[0, -1], :-1].values.flatten()]
pd.DataFrame({'sb_exp008': sb_exp008, 'ch_exp008':ch_exp008, 'mj_exp008': mj_exp008, 'final_exp008': final_exp008,'exp008': exp008})

,sb_exp008,ch_exp008,mj_exp008,final_exp008,exp008
0,0.003257,0.003257,0.004291,0.004291,0.004290
1,"0.0,","0.0,",0.000000,0.000000,0.000000
2,0.011112,0.011112,0.011497,0.011497,0.010251
3,"0.0,","0.0,",0.000000,0.000000,0.000000
4,"0.0,","0.0,",0.000000,0.000000,0.000000
5,0.037689,0.037689,0.035197,0.035197,0.036372
6,"0.0,","0.0,",0.000000,0.000000,0.000000
7,"0.0,","0.0,",0.000000,0.000000,0.000000
8,"0.0,","0.0,",0.000000,0.000000,0.000000
9,"0.0,","0.0,",0.000000,0.000000,0.000000
